In [138]:
import pandas as pd
import numpy as np
df = pd.read_csv('faturamento_clientes_alvo.csv', delimiter="|", dtype=object)

### Dados importados do banco com as seguintes informações:
1. ano: Ano referente ao Faturamento
2. mes: Mês referente ao Faturamento
3. cliente: Nome do cliente, puxado preferencialmente da base de clientes, mas caso não encontrado, puxado da razão social
4. chave_cnpj: Primeiros 8 dígitos do CNPJ do cliente, usado como chave de identificação
5. unidade_banco: Unidade referente à emissão informada no banco de dados
6. franqueado_banco: Franqueado referente à emissão informada no banco de dados
7. vendedor: vendedor responsável pelo cliente baseado na base de clientes comercial
8. unidade_comercial: Unidade referente ao cliente de acordo com a base clientes comercial (coluna cliente)
9. franqueado_comercial: Franqueado referente ao cliente de acordo com a base clientes comercial (coluna cliente)
10. faturamento: Faturamento total daquela abertura (somado)

### Objetivo:

Determinar o faturamento mensal e anual dos clientes que deixaram de emitir com a empresa, juntamente com Unidade, Franqueado e UF dos clientes, tanto do ponto de vista operacional quanto comercial.

#### Dado adicional
Um mesmo cliente pode ter mais de uma unidade, UF, franqueado. Como o objetivo é determinar apenas os principais, a seleção será feita de acordo com o maior faturamento em um único mês.


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27289 entries, 0 to 27288
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ano                   27289 non-null  object
 1   mes                   27289 non-null  object
 2   cliente               27289 non-null  object
 3   chave_cnpj            27289 non-null  object
 4   unidade_banco         27289 non-null  object
 5   franqueado_banco      27167 non-null  object
 6   uf_origem             27286 non-null  object
 7   vendedor              26004 non-null  object
 8   unidade_comercial     26004 non-null  object
 9   franqueado_comercial  26004 non-null  object
 10  faturamento           27289 non-null  object
dtypes: object(11)
memory usage: 2.3+ MB


Como a coluna faturamento veio com separador de milhar (,), foi necessário remover e transformar em float

In [140]:
df['faturamento'] = df['faturamento'].str.replace(',', '')
df = df.astype({'faturamento': float})

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27289 entries, 0 to 27288
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ano                   27289 non-null  object 
 1   mes                   27289 non-null  object 
 2   cliente               27289 non-null  object 
 3   chave_cnpj            27289 non-null  object 
 4   unidade_banco         27289 non-null  object 
 5   franqueado_banco      27167 non-null  object 
 6   uf_origem             27286 non-null  object 
 7   vendedor              26004 non-null  object 
 8   unidade_comercial     26004 non-null  object 
 9   franqueado_comercial  26004 non-null  object 
 10  faturamento           27289 non-null  float64
dtypes: float64(1), object(10)
memory usage: 2.3+ MB


Houveram alguns casos de *franqueado_banco* em branco e *uf_origem* em branco, então foi necessário tratar:

Franqueado_origem em branco: se tratavam de locais em que a JAD é responsável direta, então foi substituido por 'JAD' o valor do franqueado_banco

uf_origem em branco: 3 casos, puxei a UF de acordo com a unidade.

In [142]:
df['franqueado_banco']=np.where(df['franqueado_banco'].isna(), 'JAD', df['franqueado_banco'] )

In [143]:
df['uf_origem'] = np.where((df['uf_origem'].isna()) & (df['unidade_comercial'] == 'CO LONDRINA 01'), 'PR', df['uf_origem'])
df['uf_origem'] = np.where((df['uf_origem'].isna()) & (df['unidade_comercial'] == 'CO BLUMENAU 01'), 'SC', df['uf_origem'])

A coluna *ano* veio com separador de milhar, removi:

Continuou como coluna de texto, top

In [144]:
df['ano'] = df['ano'].str.replace(',','')

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27289 entries, 0 to 27288
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ano                   27289 non-null  object 
 1   mes                   27289 non-null  object 
 2   cliente               27289 non-null  object 
 3   chave_cnpj            27289 non-null  object 
 4   unidade_banco         27289 non-null  object 
 5   franqueado_banco      27289 non-null  object 
 6   uf_origem             27289 non-null  object 
 7   vendedor              26004 non-null  object 
 8   unidade_comercial     26004 non-null  object 
 9   franqueado_comercial  26004 non-null  object 
 10  faturamento           27289 non-null  float64
dtypes: float64(1), object(10)
memory usage: 2.3+ MB


Foi decidido utilizar o franqueado_banco como sendo o responsável pela unidade, baseado na nossa base de clientes comercial, então peguei toda a base de referência para cruzar as informações:

In [ ]:
clientes = pd.read_csv('Clientes_10_01_SEMNI.csv', dtype=object, delimiter='|')
clientes.head()

Como só será utilizada a informação de unidade e franqueado, selecionei apenas essas 2 colunas:

In [152]:
clientes = clientes[['Unidade','Franqueado']]

In [153]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 399713
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unidade     655 non-null    object
 1   Franqueado  655 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB


PONTO IMPORTANTE:

Haviam linhas em duplicata, o que é normal então decidi remover todas em que Unidade e Franqueado eram iguais, porém haviam linhas com unidade igual porém franqueado diferente, seria interessante fazer uma investigação paralela.

Para efeito do presente estudo foram excluidas as duplicatas com base na coluna Unidade, ficando com a primeira ocorrência.

Em seguida juntou-se a tabela com os dados alvo com a tabela da base de clientes filtrada, para obter os franqueados responsáveis de acordo com a base de clientes. Merge realizado pelas colunas 'unidade_banco' e 'Unidade'

In [154]:
clientes = clientes.drop_duplicates(subset='Unidade')

In [155]:
df_merge = pd.merge(left=df, right=clientes, left_on='unidade_banco', right_on='Unidade', how='left', validate='many_to_one')

In [156]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27289 entries, 0 to 27288
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ano                   27289 non-null  object 
 1   mes                   27289 non-null  object 
 2   cliente               27289 non-null  object 
 3   chave_cnpj            27289 non-null  object 
 4   unidade_banco         27289 non-null  object 
 5   franqueado_banco      27289 non-null  object 
 6   uf_origem             27289 non-null  object 
 7   vendedor              26004 non-null  object 
 8   unidade_comercial     26004 non-null  object 
 9   franqueado_comercial  26004 non-null  object 
 10  faturamento           27289 non-null  float64
 11  Unidade               24576 non-null  object 
 12  Franqueado            24576 non-null  object 
dtypes: float64(1), object(12)
memory usage: 2.9+ MB


Existem linhas com valores em branco, tratados substituindo por ND, pois são casos que a informação não foi encontrada na nossa base de clientes

Tambem foi dropada a coluna Unidade (que serviu para o merge) e renomeada a coluna Franqueado para fazer sentido no arquivo final (Franqueado_Banco_Base)

In [157]:
df_merge = df_merge.fillna('ND')

In [92]:
df_merge = df_merge.drop(['Unidade'], axis=1)

In [97]:
df_merge = df_merge.rename({'Franqueado': 'Franqueado_Banco_Base'}, axis=1)

Transformou-se os anos em colunas, agregando pela soma dos faturamentos para todas as aberturas, em seguida criou-se uma coluna com a soma de faturamento de todos os anos

In [119]:
df_pivot = df_merge.pivot_table('faturamento', index=['cliente','chave_cnpj', 'unidade_banco', 'Franqueado_Banco_Base', 'unidade_comercial', 'franqueado_comercial', 'uf_origem'], columns='ano', aggfunc=np.sum)

In [ ]:
df_pivot.head()

In [120]:
df_pivot['soma_faturamento'] = df_pivot[['2018','2019','2020','2021']].sum(axis=1)

In [121]:
df_pivot_reset = df_pivot.reset_index()

In [122]:
df_pivot_reset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7452 entries, 0 to 7451
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cliente                7452 non-null   object 
 1   chave_cnpj             7452 non-null   object 
 2   unidade_banco          7452 non-null   object 
 3   Franqueado_Banco_Base  7452 non-null   object 
 4   unidade_comercial      7452 non-null   object 
 5   franqueado_comercial   7452 non-null   object 
 6   uf_origem              7452 non-null   object 
 7   vendedor               7452 non-null   object 
 8   2018                   3083 non-null   float64
 9   2019                   2952 non-null   float64
 10  2020                   3030 non-null   float64
 11  2021                   1126 non-null   float64
 12  soma_faturamento       7452 non-null   float64
dtypes: float64(5), object(8)
memory usage: 757.0+ KB


Para ficar com um agregado do cliente, pegou-se, dentre todas as aberturas, as linhas de maior faturamento total (agregado dos 4 anos)

In [123]:
df_grouped = df_pivot_reset.groupby('cliente')['soma_faturamento'].max()
df_grouped = df_grouped.reset_index()
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cliente           773 non-null    object 
 1   soma_faturamento  773 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.2+ KB


In [158]:
df_linhas = pd.merge(df_pivot_reset, df_grouped, on=['soma_faturamento','cliente'])
df_linhas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 773 entries, 0 to 772
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cliente                773 non-null    object 
 1   chave_cnpj             773 non-null    object 
 2   unidade_banco          773 non-null    object 
 3   Franqueado_Banco_Base  773 non-null    object 
 4   unidade_comercial      773 non-null    object 
 5   franqueado_comercial   773 non-null    object 
 6   uf_origem              773 non-null    object 
 7   vendedor               773 non-null    object 
 8   2018                   431 non-null    float64
 9   2019                   439 non-null    float64
 10  2020                   437 non-null    float64
 11  2021                   281 non-null    float64
 12  soma_faturamento       773 non-null    float64
dtypes: float64(5), object(8)
memory usage: 84.5+ KB


Manteve-se apenas a abertura principal (Uma linha para cada cliente) com as informações relevantes e em seguida, para cada cliente e cada ano, juntou-se o faturamento correspondente, formando o arquivo final em que vê-se:

O faturamento anual de cada um dos clientes que tiveram um faturamento mensal maior do que 10k em algum momento de Jan-2018 a Out-2021 e não fizeram nenhuma emissão com a Jad entre Nov-2021 a 15-Jan-2022.

In [129]:
df_describe = df_linhas[['cliente','chave_cnpj','uf_origem', 'unidade_banco', 'Franqueado_Banco_Base', 'unidade_comercial', 'franqueado_comercial']]
df_describe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 773 entries, 0 to 772
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cliente                773 non-null    object
 1   chave_cnpj             773 non-null    object
 2   uf_origem              773 non-null    object
 3   unidade_banco          773 non-null    object
 4   Franqueado_Banco_Base  773 non-null    object
 5   unidade_comercial      773 non-null    object
 6   franqueado_comercial   773 non-null    object
 7   vendedor               773 non-null    object
dtypes: object(8)
memory usage: 54.4+ KB


In [130]:
df_total_clientes =df.pivot_table('faturamento', index=['cliente'], columns='ano', aggfunc=np.sum)
df_total_clientes = df_total_clientes.reset_index()

In [ ]:
df.info()

In [ ]:
df_b = df.copy()
df_b['mes'] = df_b['mes'].astype(int)
df_total_clientes_am =df_b.pivot_table('faturamento', index=['cliente'], columns=['ano','mes'], aggfunc=np.sum)
df_total_clientes_am = df_total_clientes_am.reset_index()
df_total_clientes_am.head()

In [ ]:
df_final = pd.merge(left=df_describe, right=df_total_clientes, on='cliente')
df_final_am = pd.merge(left=df_describe, right=df_total_clientes_am, on='cliente')

In [ ]:
df_final_am.info()
df_final.to_excel('Estudo_Final.xlsx', index=False)

df_final_am.to_excel('Estudo_Final_ANO_e_MES.xlsx', index=False)